<a href="https://colab.research.google.com/github/julianjara/machine_learning/blob/master/my_dataset_Julian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Julian Jaramillo

this is the solution to builidng tree   
different from the video

# How a Decision Tree Works

Here we prepare the sport data we have used as the example.

In [1]:
import pandas as pd
import numpy as np

_raw_data = """Sky,Humidity,Wind,Play
Sunny,High,Low,No
Sunny,High,High,No
Overcast,High,Low,Yes
Rain,High,Low,Yes
Rain,Low,Low,Yes
Rain,Low,High,No
Overcast,Low,High,Yes
Sunny,High,Low,No
Sunny,Low,Low,Yes
Rain,Low,Low,Yes
Sunny,Low,High,Yes
Overcast,High,High,Yes
Overcast,Low,Low,Yes
Rain,High,High,No
"""
with open("sport_data.csv", "w") as f:
    f.write(_raw_data)
df = pd.read_csv("sport_data.csv")

 Here we use the entropy computation function we built from the last time to measure the information in the `play` attribute of the data.

In [2]:
def compute_entropy(y):
    """
    :param y: The data samples of a discrete distribution
    """
    if len(y) < 2: #  a trivial case
        return 0
    freq = np.array( y.value_counts(normalize=True) )
    return -(freq * np.log2(freq + 1e-6)).sum() # the small eps for 
    # safe numerical computation 

# Build a Decision Tree

We continue to use  the data  structure of  decision trees which we have studied in the last time.

Recall from the last time: we have manually constructed the tree structure by creating and assigning values to the children tree nodes --  the operation is recursive after we have done the children nodes of one tree node,  we will continue work with the newly created nodes and have more descendants in the tree.  So eventually we will have a tree hierarchy.

```python
dt = Tree()
dt.root.split_feat_name = "Outlook"
dt.root.children = {"Sunny":TreeNode(), "Overcast":TreeNode(), "Rain":TreeNode()}
dt.root.children["Sunny"].split_feat_name = "Humidity"
dt.root.children["Sunny"].children = {"High":TreeNode(), "Normal":TreeNode()}
dt.root.children["Sunny"].children["High"].decision = "No"
dt.root.children["Sunny"].children["Normal"].decision = "Yes"
dt.root.children["Overcast"].decision = "Yes"
```

For the next, let us make the tree construction process automatic. We will make use of the data structure of the tree nodes defined above.

__Exercise__

Into next code block you are required to do two exercises:

1. to complete the decision-making process
2. to complete the tree-node construction process


In [4]:
def compute_info_gain(samples, attr, target):
    values = samples[attr].value_counts(normalize=True)
    split_ent = 0
    for v, fr in values.iteritems():
        index = samples[attr]==v
        sub_ent = compute_entropy(target[index])
        split_ent += fr * sub_ent
    
    ent = compute_entropy(target)
    return ent - split_ent

class TreeNode:
    """
    A recursively defined data structure to store a tree.
    Each node can contain other nodes as its children
    """
    def __init__(self):
        self.children = {} # Sub nodes --
        # recursive, those elements of the same type (TreeNode)
        self.decision = None # Undecided
        self.split_feat_name = None # Splitting feature

    def pretty_print(self, prefix=''):
        if self.split_feat_name is not None:
            for k, v in self.children.items():
                v.pretty_print(f"{prefix}:When {self.split_feat_name} is {k}")
                #v.pretty_print(f"{prefix}:{k}:")
        else:
            print(f"{prefix}:{self.decision}")

    def predict(self, sample):
        if self.decision is not None:
            # uncomment to get log information of code execution
            print("Decision:", self.decision)
            return self.decision
        else: 
            # this node is an internal one, further queries about an attribute 
            # of the data is needed.
            attr_val = sample[self.split_feat_name]
            child = self.children[attr_val]
            # uncomment to get log information of code execution
            print("Testing ", self.split_feat_name, "->", attr_val)

            # [Exercise]
            # Insert your code here
            return child.predict(sample)

    def fit(self, X, y):
        """
        The function accepts a training dataset, from which it builds the tree 
        structure to make decisions or to make children nodes (tree branches) 
        to do further inquiries
        :param X: [n * p] n observed data samples of p attributes
        :param y: [n] target values
        """
        if len(X) == 0:
            # If the data is empty when this node is arrived, 
            # we just make an arbitrary decision
            self.decision = "Yes"
            return
        else: 
            unique_values = y.unique()
            if len(unique_values) == 1:
                # [Exercise]
                # Insert your code here
                self.decision = unique_values[0]
                return
            else:
                info_gain_max = 0
                for a in X.keys(): # Examine each attribute
                    aig = compute_info_gain(X, a, y)
                    if aig > info_gain_max:
                        # [Exercise]
                        # Insert your code here
                        ig_max = aig
                        self.split_feat_name = a
                print(f"Split by {self.split_feat_name}, IG: {ig_max:.2f}")
                self.children = {}
                for v in X[self.split_feat_name].unique():
                    index = X[self.split_feat_name] == v
                    self.children[v] = TreeNode()
                    # [Exercise]
                    # Insert your code here
                    self.children[v].fit(X[index], y[index])

# Test tree building
attrs = ['Sky', 'Humidity', 'Wind']
data = df[attrs]
target = df["Play"]

t = TreeNode()
t.fit(data, target)

Split by Wind, IG: 0.05
Split by Humidity, IG: 0.31
Split by Sky, IG: 1.00
Split by Humidity, IG: 0.08
Split by Sky, IG: 0.92
Split by Sky, IG: 0.92


In [5]:
# Test tree working
for i in [0,2,4]:
    print(f"Test predict sample[{i}]: \n{data.iloc[i]}\n\tTarget: {target.iloc[i]}")
    print(f"Making decision ...")
    pred = t.predict(data.iloc[i])

Test predict sample[0]: 
Sky         Sunny
Humidity     High
Wind          Low
Name: 0, dtype: object
	Target: No
Making decision ...
Testing  Wind -> Low
Testing  Humidity -> High
Testing  Sky -> Sunny
Decision: No
Test predict sample[2]: 
Sky         Overcast
Humidity        High
Wind             Low
Name: 2, dtype: object
	Target: Yes
Making decision ...
Testing  Wind -> Low
Testing  Humidity -> High
Testing  Sky -> Overcast
Decision: Yes
Test predict sample[4]: 
Sky         Rain
Humidity     Low
Wind         Low
Name: 4, dtype: object
	Target: Yes
Making decision ...
Testing  Wind -> Low
Testing  Humidity -> Low
Decision: Yes
